In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_excel('/content/Dry_Bean_Dataset.xlsx')


In [7]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [9]:
x=torch.tensor(X.values,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.long)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
from torch.utils.data import Dataset
class customdataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y
  def __len__(self):
    return self.X.shape[0]
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [12]:
train_dataset=customdataset(X_train,y_train)
val_dataset=customdataset(X_val,y_val)
test_dataset=customdataset(X_test,y_test)

In [13]:
batch_size = 16

In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [26]:
input_size = 16  # Assuming input features of size 16 for example
hidden_size = 10    # Number of neurons in the hidden layer
output_size = 7     # Number of classes in the output layer
learning_rate = 0.001
batch_size = 16
epochs = 15

In [27]:
model = MLP(input_size, hidden_size, output_size)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation loop (optional)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        total_loss = 0.0
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            total_loss += val_loss.item()

        average_val_loss = total_loss / len(val_loader)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss:.4f}')

# Test the model on the test set
model.eval()
with torch.no_grad():
    total_loss = 0.0
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        test_loss = criterion(test_outputs, test_labels)
        total_loss += test_loss.item()

    average_test_loss = total_loss / len(test_loader)
    print(f'Test Loss: {average_test_loss:.4f}')

Epoch 1/15, Validation Loss: 18.3916
Epoch 2/15, Validation Loss: 17.5478
Epoch 3/15, Validation Loss: 9.4450
Epoch 4/15, Validation Loss: 21.4331
Epoch 5/15, Validation Loss: 7.8433
Epoch 6/15, Validation Loss: 18.5379
Epoch 7/15, Validation Loss: 8.3181
Epoch 8/15, Validation Loss: 10.8037
Epoch 9/15, Validation Loss: 10.7717
Epoch 10/15, Validation Loss: 7.5443
Epoch 11/15, Validation Loss: 4.6639
Epoch 12/15, Validation Loss: 8.4376
Epoch 13/15, Validation Loss: 4.2630
Epoch 14/15, Validation Loss: 9.9752
Epoch 15/15, Validation Loss: 3.7104
Test Loss: 3.6235


In [30]:
model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, predicted = torch.max(test_outputs, 1)
        total_correct += (predicted == test_labels).sum().item()
        total_samples += test_labels.size(0)

    test_accuracy = total_correct / total_samples
    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 49.54%
